## QAT 概述

* https://zhuanlan.zhihu.com/p/489995404#

QAT 概述
与其他量化方法相比，QAT 在 训练过程中 模拟量化的效果，可以获得更高的 accuracy。在训练过程中，所有的计算都是在浮点上进行的，使用 fake_quant 模块通过夹紧和舍入的方式对量化效果进行建模，模拟 INT8 的效果。模型转换后，权值和激活被量化，激活在可能的情况下被融合到前一层。它通常与 CNN 一起使用，与 PTQ 相比具有更高的 accuracy。

In [12]:
import torch
import torchvision

print(f'torch: {torch.__version__} \n'
      f'torchvision: {torchvision.__version__}')

torch: 1.10.1 
torchvision: 0.11.2


In [5]:

# 定义简单的浮点模块
from torch import nn, Tensor


class M(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = torch.nn.Conv2d(1, 1, 1)
        self.relu = torch.nn.ReLU()

    def _forward_impl(self, x: Tensor) -> Tensor:
        '''提供便捷函数'''
        x = self.conv(x)
        x = self.relu(x)
        return x

    def forward(self, x: Tensor) -> Tensor:
        x= self._forward_impl(x)
        return x
    
# from torch.ao.quantization import QuantStub, DeQuantStub
from torch.quantization import QuantStub, DeQuantStub

# ## 定义可量化模块
# 将浮点模块 `M` 转换为可量化模块 `QM`（量化流程的最关键的一步）。
class QM(M):
    '''
    Args:
        is_print: 为了测试需求，打印一些信息
    '''
    def __init__(self, is_print: bool=False):
        super().__init__()
        self.is_print = is_print
        self.quant = QuantStub() # 将张量从浮点转换为量化
        self.dequant = DeQuantStub() # 将张量从量化转换为浮点

    def forward(self, x: Tensor) -> Tensor:
        # 手动指定张量将在量化模型中从浮点模块转换为量化模块的位置
        x = self.quant(x)
        if self.is_print:
            print('量化前的类型：', x.dtype)
        x = self._forward_impl(x)
        if self.is_print:
            print('量化中的类型：',x.dtype)
        # 在量化模型中手动指定张量从量化到浮点的转换位置
        x = self.dequant(x)
        if self.is_print:
            print('量化后的类型：', x.dtype)
        return x

## 量化模型
定义比 `M` 稍微复杂一点的浮点模块：

In [6]:
class M2(M):
    def __init__(self):
        super().__init__()
        self.bn = torch.nn.BatchNorm2d(1)

    def _forward_impl(self, x: Tensor) -> Tensor:
        '''提供便捷函数'''
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


In [7]:
# 同样需要定义可量化模块：

class QM2(M2, QM):
    def __init__(self):
        super().__init__()

In [8]:
# 创建浮点模型实例：

# 创建模型实例
model_fp32 = QM2()
model_fp32
# QM2(
#   (conv): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
#   (relu): ReLU()
#   (quant): QuantStub()
#   (dequant): DeQuantStub()
#   (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
# )

QM2(
  (conv): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
  (relu): ReLU()
  (quant): QuantStub()
  (dequant): DeQuantStub()
  (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
# 模型必须设置为训练模式，以便 QAT 可用：
model_fp32.train();
# 添加量化配置（与 PTQ 相同相似）：
model_fp32.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')



# 融合 QAT 模块
QAT 的模块融合与 PTQ 相同相似：

In [15]:
from torch.quantization import fuse_modules_qat

model_fp32_fused = fuse_modules_qat(model_fp32,
                                    [['conv', 'bn', 'relu']])

ImportError: cannot import name 'fuse_modules_qat' from 'torch.quantization' (/home/chongqinghuang/anaconda3/envs/waymo_38/lib/python3.8/site-packages/torch/quantization/__init__.py)

## 准备 QAT 模型
这将在模型中插入观测者和伪量化模块，它们将在校准期间观测权重和激活的张量。

In [ ]:
model_fp32_prepared = torch.quantization.prepare_qat(model_fp32_fused)


## 训练 QAT 模型
# 下文会编写实际的例子，此处没有显示
training_loop(model_fp32_prepared)

将观测到的模型转换为量化模型。需要：

* 量化权重，计算和存储用于每个激活张量的尺度（scale）和偏差（bias）值，
* 在适当的地方融合模块，并用量化实现替换关键算子。

In [ ]:
model_fp32_prepared.eval()
model_int8 = torch.quantization.convert(model_fp32_prepared)

In [ ]:
# 运行模型，相关的计算将在 {data}torch.qint8 中发生。

res = model_int8(input_fp32)
